In [8]:
#coding: utf-8
import warnings
warnings.filterwarnings("ignore")
import os
import time
import random
import jieba
import sklearn
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import matplotlib.pyplot as plt
from numpy import vstack
from sklearn.feature_extraction.text import TfidfTransformer

if __name__ == '__main__':
    #train_data = pd.read_csv('./train/train_data.txt',sep='\t',header=None)
    #train_label = pd.read_csv('./train/train_labels.txt',sep='\t',header=None)
    with open('./train/train_data.txt','r') as fr:
        train_data_list = [line.strip().split('\t') for line in fr.readlines()]
    with open('./train/train_labels.txt','r') as fr:
        train_label_list = [line.strip().split('\t') for line in fr.readlines()]
        # predict
    with open('./test/test_data.txt','r') as fr:
        test_list = [line.strip().split('\t') for line in fr.readlines()]


In [9]:
#get it to right countV format
train_feature_list = [" ".join(x) for x in train_data_list ]
train_label_list = [" ".join(x) for x in train_label_list]
test_list = [" ".join(x) for x in test_list ]

In [10]:
#cross validation 
def cos_vali(data_list,class_list,test_size = 0.2):
    data_class_list = list(zip(data_list, class_list))
    random.shuffle(data_class_list)
    index = int(len(data_class_list)*test_size)#+1
    train_list = data_class_list[index:]
    test_list = data_class_list[:index]
    train_data_list, train_class_list = zip(*train_list)
    test_data_list, test_class_list = zip(*test_list)
    return train_data_list, test_data_list, train_class_list, test_class_list

In [11]:
# split the dataset
train_data_list, test_data_list, train_class_list, test_class_list = cos_vali(train_feature_list,train_label_list,test_size = 0.2)

In [12]:
# extract the feature
vectorizer = CountVectorizer(max_df=0.85,max_features=1000)

tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf = tfidf_transformer.fit_transform(vectorizer.fit_transform(train_data_list))
tf_idf_vector=tfidf_transformer.transform(vectorizer.transform(test_data_list))

pred_vect = tfidf_transformer.transform(vectorizer.transform(test_list))

In [13]:
# text clf
def TextClassifier(train_feature_list, test_feature_list, 
                    train_class_list, test_class_list,test_list):
    """
    函数说明:分类器
    Parameters:
        train_feature_list - 训练集向量化的特征文本
        test_feature_list - 测试集向量化的特征文本
        train_class_list - 训练集分类标签
        test_class_list - 测试集分类标签
        test_list       - 要预测的
    Returns:
        test_accuracy - 分类器精度
    """
    #train_feature_list = list(train_feature_list)
    train_class_list = list(train_class_list)
    #test_feature_list = list(test_feature_list)
    test_class_list = list(test_class_list)

    X_train = train_feature_list
    Y_train  = train_class_list

    X_train_c = np.copy(train_feature_list)
    Y_train_c  = np.copy(train_class_list)
    
    X_val  = test_feature_list
    Y_val = test_class_list

    len_X_train = len(X_train)
    len_X_val = len(X_val)

    X = vstack([X_train,X_val])
    X = np.array(X)
    Y_train.extend(Y_val)
    Y = np.array(Y_train)

    #Mark the training-validation splits
    train_i = np.ones((len_X_train,), dtype = int) * -1
    valid_i = np.zeros((len_X_val,), dtype = int)
    split_fold = np.concatenate((train_i, valid_i))
    ps = PredefinedSplit(split_fold)
    
    params = {'alpha':np.linspace(0.0001,1,10000)}
    classifier = MultinomialNB()
    
    param_search = GridSearchCV(classifier,
                            params, 
                    scoring=metrics.make_scorer(metrics.f1_score, average='macro'),
                                cv=ps,
                                return_train_score=True)
    param_search.fit(X,Y)
    results = param_search.cv_results_
    best_params = param_search.best_params_ 
    
    clf = MultinomialNB(alpha = best_params['alpha'])
    clf.fit(X_train_c,Y_train_c)
    #validation
    Y_pred = clf.predict(X_val)
    vali_accuracy = metrics.f1_score(Y_val, Y_pred, average='macro')
    
    pred_target = clf.predict(test_list)
    
    return vali_accuracy,pred_target


In [15]:
vali_accuracy, pred_target = TextClassifier(tfidf.toarray().tolist(), tf_idf_vector.toarray().tolist(),train_class_list, test_class_list,pred_vect.toarray().tolist())

#np.savetxt("test_v2.txt", pred_target,fmt = '%d')
pred_target
print(vali_accuracy)

KeyboardInterrupt: 

In [23]:
a = '1'
ord('0')

48

In [24]:
a = []
for i in pred_target:
    a.append([ord(i)-48])

In [27]:
np.savetxt("test_v2.txt", a,fmt = '%d')

In [25]:
a

[[0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1]

In [17]:
vali_accuracy

0.840772256157913

In [ ]:

pred_vector=tfidf_transformer.transform(vectorizer.transform(test_feature_list))


In [1]:
import winsound
winsound.Beep(600,1000)

ModuleNotFoundError: No module named 'winsound'

In [23]:
len(train_data_list)
len(test_data_list)

5000

In [ ]:
'''
    clf = tree.DecisionTreeClassifier(max_depth = None)
    clf = clf.fit(TrainSet,train_target)

    true_test_data = pd.read_csv('./test/test_data.txt',sep='\t',header=None)
    with open('./test/test_data.txt','r') as fr:
        test_data_list = [line.strip().split('\t') for line in fr.readlines()]
    test_data_dict = {}
    for i in range(true_test_data.index.size):
        for item in [int(s) for s in test_data_list[i][0].split() if s.isdigit()]:
            vector[item] = 1
        test_data_dict[i]=[]
        test_data_dict[i].extend(vector)
        vector = [0 for i in range(10000)]
    #true_TestSet = pd.DataFrame(train_data_dict).T
    true_TestSet = pd.DataFrame(train_data_dict)
    pred_target = []
    for i in range(true_test_data.index.size):
        #m = clf.predict([TestSet.iloc[i,:]]).tolist()
        m = clf.predict([true_TestSet.iloc[:,i]]).tolist()
        pred_target.extend(m)
    np.savetxt("test_v2.txt", pred_target,fmt = '%d')
    print('done!')
'''
